<?php
App::uses('Controller', 'Controller');

/**
 * Class TestingController
 * @property  TtgPaymentHistory $TtgPaymentHistory
 */
class TestingController extends Controller
{
    public $uses = ['TtgPaymentHistory'];

    public $testGroups
        = [
            382   => 'jaytest6',
            26819 => 'prodtest9',
            25302 => 'prodtest5',
            25644 => 'prodtest8',
            19    => 'Tapatalk Support Group',
            25606 => 'prodtest7',
            33    => 'Tapatest',
            25058 => 'prodtest4',
            22099 => 'prodtest1',
            25    => 'Tapatest Support Forum',
            41037 => 'prodtest12',
            26820 => 'prodtest10',
            25591 => 'prodtest6',
            41589 => 'summer9',
            22078 => 'summerlu',
            22080 => 'summer01',
            41814 => 'summer02',
            41575 => 'yltest1',
        ];

    public function ttgPaymentReport()
    {
        $model         = $this->TtgPaymentHistory;
        $sql
                       = <<<EOF
SELECT p.*, f.name, f.url FROM ttg_payment_history p 
LEFT JOIN ttg_goldpoint_history g ON p.id=g.ph_id 
LEFT JOIN forums f ON f.id=g.fid 
WHERE target = 'donate' AND p.money <> 0 AND f.name NOT LIKE 'prodtest%' AND f.name NOT LIKE 'jaytest%' AND p.forum_id NOT IN(19,25,33,41589,22078,22080,41814,41575)
ORDER BY p.forum_id ASC, p.created_at DESC
EOF;
        $data          = $model->query($sql);
        $group         = [];
        $countAll      = 0;
        $countStripe   = 0;
        $countPayPal   = 0;
        $countOnetime  = 0;
        $countPerMonth = 0;

        foreach ($data as $key => $value)
        {
            $forum_id = $value['p']['forum_id'];
            if (!isset($group[$forum_id]))
            {
                $group[$forum_id] = [];
            }

            if (!isset($group[$forum_id]['forum_id']))
            {
                $group[$forum_id]['forum_id'] = $forum_id;
            }

            if (!isset($group[$forum_id]['name']))
            {
                $group[$forum_id]['name'] = $value['f']['name'];
            }

            if (!isset($group[$forum_id]['url']))
            {
                $group[$forum_id]['url'] = $value['f']['url'];
            }

            $group[$forum_id]['data'][] = $value['p'];

            if (!isset($group[$forum_id]['total']))
            {
                $group[$forum_id]['total'] = 0;
            }

            $group[$forum_id]['total'] += $value['p']['money'];

            if ($value['p']['platform'] == 'stripe')
            {
                $countStripe++;
            }
            else
            {
                $countPayPal++;
            }

            if ($value['p']['type'] == 'subscribe')
            {
                $countPerMonth++;
            }
            else
            {
                $countOnetime++;
            }

            $countAll++;
        }

        foreach ($group as $key => $value)
        {
            foreach ($value['data'] as $k => $v)
            {
                $group[$key]['weekly'][date('W', strtotime($v['created_at']))]['data'][] = $v;
            }
        }

        foreach ($group as $key => $value)
        {
            unset($group[$key]['data']);
            foreach ($value['weekly'] as $k1 => $v1)
            {
                foreach ($v1['data'] as $k2 => $v2)
                {
                    $group[$key]['weekly'][$k1]['money'][]    = $v2['money'];
                    $group[$key]['weekly'][$k1]['platform'][] = $v2['platform'];
                    $group[$key]['weekly'][$k1]['type'][]     = $v2['type'];
                }
            }
        }


        foreach ($group as $key => $value)
        {
            foreach ($value['weekly'] as $k1 => $v1)
            {
                foreach ($v1['data'] as $k2 => $v2)
                {
                    $group[$key]['weekly'][$k1]['date']  = $this->getStartAndEndDate($k1, 2017);
                    $group[$key]['weekly'][$k1]['total'] = number_format(array_sum($group[$key]['weekly'][$k1]['money']), 2);

                    $platformTotal = count($group[$key]['weekly'][$k1]['platform']);
                    $platformCount = array_count_values($group[$key]['weekly'][$k1]['platform']);
                    $stripe        = $platformCount['stripe'];
                    $paypal        = $platformCount['paypal'];

                    $group[$key]['weekly'][$k1]['platform_percentage'] = [
                        'stripeFraction'   => $stripe ? $stripe . '/' . $platformTotal : 0,
                        'paypalFraction'   => $paypal ? $paypal . '/' . $platformTotal : 0,
                        'stripePercentage' => $stripe ? number_format(($stripe / $platformTotal) * 100, 2) . '%' : '0.00%',
                        'paypalPercentage' => $paypal ? number_format(($paypal / $platformTotal) * 100, 2) . '%' : '0.00%',
                    ];

                    $typeTotal = count($group[$key]['weekly'][$k1]['type']);
                    $typeCount = array_count_values($group[$key]['weekly'][$k1]['type']);
                    $onetime   = $typeCount['donate'];
                    $subscribe = $typeCount['subscribe'];

                    $group[$key]['weekly'][$k1]['period'] = [
                        'onetimeFraction'     => $onetime ? $onetime . '/' . $typeTotal : 0,
                        'subscribeFraction'   => $subscribe ? $subscribe . '/' . $typeTotal : 0,
                        'onetimePercentage'   => $onetime ? number_format(($onetime / $typeTotal) * 100, 2) . '%' : '0.00%',
                        'subscribePercentage' => $subscribe ? number_format(($subscribe / $typeTotal) * 100, 2) . '%' : '0.00%',
                    ];
                }
            }
        }

        function cmp($a, $b)
        {
            if ($a['total'] == $b['total'])
            {
                return 0;
            }
            return ($a['total'] > $b['total']) ? -1 : 1;
        }

        usort($group, "cmp");

        echo <<<EOF
<style> 
table{border-collapse: collapse;}
th,td{min-width:50px;height:25px;}
table,th,tr,td{border: 1px solid #CCCCCC;}
</style>
EOF;

        echo "<table><tbody><thead><th></th><th>name</th><th>url</th>";

        $startWeek = 39;
        $endWeek   = 49;

        for ($j = $startWeek; $j <= $endWeek; $j++)
        {
            $date = $this->getStartAndEndDate($j, 2017);
            echo "<th>";
            echo "<p>" . $date[0] . "</p>";
            echo "<p>" . $date[1] . "</p>";
            echo "</th>";
        }

        echo "<th>total</th>";

        echo "</thead>";

        $allTotal    = 0;
        $weeklyTotal = [];

        foreach ($group as $key => $value)
        {
            echo "<tr>";

            echo "<td>" . $value['forum_id'] . "</td>";
            echo "<td>" . $value['name'] . "</td>";
            echo "<td>" . $value['url'] . "</td>";

            for ($i = $startWeek; $i <= $endWeek; $i++)
            {
                echo "<td>";

                if (isset($value['weekly'][$i]))
                {
                    echo "<p>" . $value['weekly'][$i]['total'] . "</p>";
//                    echo "<p>" .
//                         'stripe:' . $value['weekly'][$i]['platform_percentage']['stripeFraction'] . $value['weekly'][$i]['platform_percentage']['stripePercentage'] .
//                         'paypal:' . $value['weekly'][$i]['platform_percentage']['paypalFraction'] . $value['weekly'][$i]['platform_percentage']['paypalPercentage'] .
//                         "</p>";
//                    echo "<p>" .
//                         'oneTime:' . $value['weekly'][$i]['period']['onetimeFraction'] . $value['weekly'][$i]['period']['onetimePercentage'] .
//                         'per month:' . $value['weekly'][$i]['period']['subscribeFraction'] . $value['weekly'][$i]['period']['subscribePercentage'] .
//                         "</p>";

                    if (!isset($weeklyTotal[$i]))
                    {
                        $weeklyTotal[$i] = 0;
                    }

                    $weeklyTotal[$i] += $value['weekly'][$i]['total'];
                }

                echo "</td>";
            }


            echo "<td>" . number_format($value['total'], 2) . "</td>";
            $allTotal += $value['total'];

            echo "</tr>";
        }

        echo "<tfooter><tr>";
        echo "<td></td>";
        echo "<td>stripe:" . number_format(($countStripe / $countAll) * 100, 2) . "% paypal:" . number_format(($countPayPal / $countAll) * 100, 2) . "%</td>";
        echo "<td>one time:" . number_format(($countOnetime / $countAll) * 100, 2) . "% per month:" . number_format(($countPerMonth / $countAll) * 100, 2) . "%</td>";

        for ($j = $startWeek; $j <= $endWeek; $j++)
        {
            echo "<td>" . number_format($weeklyTotal[$j], 2) . "</td>";
        }

        echo "<td>" . number_format($allTotal, 2) . "</td></tr></tfooter></tbody></table>";

        exit();
    }


    private function getStartAndEndDate($week, $year)
    {
        $dto       = new DateTime();
        $result[0] = $dto->setISODate($year, $week, 0)->format('m-d');
        $result[1] = $dto->setISODate($year, $week, 6)->format('m-d');
        return $result;
    }

}